### Autograd

In [124]:
# function to calculate the derivative of y = x^2
def dy_dx(x):
  return 2*x

In [125]:
dy_dx(3)

6

In [126]:
import math

# function to calculate the derivative of y = x^2 * cos(x^2)
def dz_dx(x):
  return 2 * x * math.cos(x**2) # derivative of x^2 * cos(x^2)

In [127]:
dz_dx(2)

-2.6145744834544478

### Example 1
#### y = x^2

In [128]:
# calculating the derivative using PyTorch
import torch

In [129]:
x = torch.tensor(3.0, requires_grad=True) # scalar value 3.0 # required grad is set to True to track the gradient
x


tensor(3., requires_grad=True)

In [130]:
y = x**2 # 3^2 = 9
y

tensor(9., grad_fn=<PowBackward0>)

In [131]:
y.backward() # backward pass
y 

tensor(9., grad_fn=<PowBackward0>)

In [132]:
x.grad

tensor(6.)

### Example 2 
#### y = x^2, z = sin(y)

In [133]:
x = torch.tensor(3.0, requires_grad=True)
x

tensor(3., requires_grad=True)

In [134]:
y = x**2 # y = x^2
y

tensor(9., grad_fn=<PowBackward0>)

In [135]:
z = torch.sin(y) # z = sin(y) trigonometric function
z

tensor(0.4121, grad_fn=<SinBackward0>)

In [136]:
z.backward() # backward pass

In [137]:
x.grad

tensor(-5.4668)

### Example 3
#### Neural Network

In [138]:
import torch

# inputs
x = torch.tensor(6.7) # input feature
y = torch.tensor(0.0) # output label (binary)

w = torch.tensor(1.0) # weight
b = torch.tensor(0.0) # bias

In [139]:
# Binary Cross Entropy Loss
def binary_cross_entropy(prediction, target):
  epsilon = 1e-8 # to avoid log(0)
  prediction = torch.clamp(prediction, epsilon, 1-epsilon) # clamping the values between epsilon and 1-epsilon
  return -target*torch.log(prediction) + (1-target)*torch.log(1-prediction) # binary cross entropy loss

In [140]:
# forward pass
z = w * x + b # weighted sum
y_pred = torch.sigmoid(z) # sigmoid function

# Computing binary cross entropy loss
loss = binary_cross_entropy(y_pred, y)
loss

tensor(-6.7012)

In [141]:
# backward pass
# Derivative of binary cross entropy loss
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y) / (y_pred * (1 - y_pred)) # derivative of binary cross entropy loss with respect to y_pred

# 2. dy_pred/dz: Prediction (y_pred) with respect to the weighted sum (z) sigmoid function
dy_pred_dz = y_pred * (1 - y_pred) # derivative of y_pred with respect to z

# 3. dz/dw and dz/db: Weighted sum (z) with respect to the weight (w) and bias (b)
dz_dw = x # derivative of z with respect to w
dz_db = 1 # derivative of z with respect to b (bias)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [142]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


In [143]:
# Using Autograd for all the manual computations
x = torch.tensor(6.7) # input feature
y = torch.tensor(0.0) # output label (binary)

In [144]:
w = torch.tensor(1.0, requires_grad=True) # weight
b = torch.tensor(0.0, requires_grad=True) # bias

In [145]:
# forward pass
z = w * x + b # weighted sum
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [146]:
# calculating sigmoid
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [147]:
# calculating loss
loss = binary_cross_entropy(y_pred, y)
loss

tensor(-6.7012, grad_fn=<AddBackward0>)

In [148]:
# backward pass
loss.backward()

In [149]:
print(w.grad) # gradient of loss w.r.t weight
print(b.grad) # gradient of loss w.r.t bias

tensor(-6.6918)
tensor(-0.9988)


In [150]:
# for vector input tensors
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True) # input feature
x


tensor([1., 2., 3.], requires_grad=True)

In [151]:
y = (x**2).mean() # output label
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [152]:
y.backward()

In [153]:
x.grad # gradient of loss w.r.t input which will be 3 gradients

tensor([0.6667, 1.3333, 2.0000])

### Clearing Grads

In [159]:
x = torch.tensor(3.0, requires_grad=True)
x

tensor(3., requires_grad=True)

In [160]:
y = x**2 # y = x^2
y

tensor(9., grad_fn=<PowBackward0>)

In [161]:
y.backward()

In [162]:
x.grad # it will going to add whenever we do backward pass

tensor(6.)

In [163]:
# solution for clearing the gradients
x.grad.zero_()

tensor(0.)

### Disabling gradients tracking

In [164]:
x = torch.tensor(3.0, requires_grad=True)
x

tensor(3., requires_grad=True)

In [165]:
y = x**2 # y = x^2
y

tensor(9., grad_fn=<PowBackward0>)

In [166]:
y.backward()

In [167]:
x.grad

tensor(6.)

#### 1. Option 1 for disabling gradients

In [168]:
x.requires_grad = False # set the requires_grad flag to False
x

tensor(3.)

In [169]:
y = x**2 # y = x^2
y

tensor(9.)

In [171]:
# y.backward() # because of the disabling backward pass will not work

#### 2. Option 2 for disabling gradient

In [173]:
x = torch.tensor(3.0, requires_grad=True)
x

tensor(3., requires_grad=True)

In [174]:
z = x.detach() # detach the tensor from the computation graph
z

tensor(3.)

In [178]:
y = x**2 # y = x^2 # gradient function
y

tensor(9., grad_fn=<PowBackward0>)

In [177]:
y1 = z**2 # y = z^2 # no gradient function so no backward pass
y1

tensor(9.)

#### 3. Option 3 for disabling gradients

In [179]:
x = torch.tensor(3.0, requires_grad=True)
x

tensor(3., requires_grad=True)

In [180]:
with torch.no_grad():
  y = x**2 # y = x^2
  y